In [13]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import seaborn as sns
import h5py
import tqdm
import zipfile
from tqdm.notebook import tqdm_notebook
from progress.bar import IncrementalBar
import random
import pandas as pd
import time
import os
import math
from joblib import Parallel, delayed
import importlib
import imp
%matplotlib inline

In [2]:
np.set_printoptions(suppress=True)

In [3]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
cd ../src

/home/rfit/Telescope_Array/Parameters_all/src


In [5]:
import reconstruction as reco

In [6]:
param_names=['signal','pl_fr','real_wf-pl_fr','mask']

In [7]:
def norming(data,param_names=param_names,log=True):
    
    # ADD LOGARIFM FOR SIGNAL
    norm_params={}
    for i in range(data.shape[-1]):
        if (i==0 and log):
            data[:,:,:,i]=tf.math.log(data[:,:,:,i]+1)
        max_c=data[:,:,:,i].max()
        min_c=data[:,:,:,i].min()
        mean_c=data[:,:,:,i].mean()
        std_c=data[:,:,:,i].std()
        print('max_c,min_c',max_c,min_c)
        data[:,:,:,i]=(data[:,:,:,i]-min_c)/(max_c-min_c)

        norm_params[param_names[i]]=np.array([max_c,min_c])
    return norm_params

In [8]:
path_old='../../data_01_24.h5'
path_new = '/home3/ivkhar/TA/data/MC/bundled/pr_q3_14yr_1745_0010_excl-sat_F_excl-geo_F_take-log-wf-False_bundled.h5'
path = path_new
num=-1
with h5py.File(path,'r') as f:
    print(f.keys())
    data=f['dt_bunlde'][:num,:,:,3:7]
    print(data.shape)
    detectors_rub = f['dt_bunlde'][:num,:,:,:3] * 1.2 #/ 6 # norming
    real_ang = f['mc_params'][:num,4:6]
    recos = f['recos'][:num]
    dt_params =f['dt_params'][:num]
    ev_starts = f['ev_starts'][:num]
    if path == path_new:
        dt_bunlde_mask = f['dt_bunlde_mask'][:num]
norm_params=norming(data,log=False)
theta = tf.cast(real_ang[:,0:1]/180*3.1415,tf.float32)
phi = tf.cast(real_ang[:,1:2]/180*3.1415,tf.float32)
courve  = tf.cast(recos[:,6:7],tf.float32)
S800 = tf.cast(recos[:,2:3],tf.float32)
chi_rub = tf.cast(recos[:,5:6],tf.float32)

<KeysViewHDF5 ['det_max_params', 'det_max_wf', 'dt_bunlde', 'dt_bunlde_mask', 'dt_mask', 'dt_params', 'ev_ids', 'ev_starts', 'mc_params', 'recos', 'wfs_flat']>
(173043, 6, 6, 4)
max_c,min_c 8387.6 0.0
max_c,min_c 11.0 -11.551
max_c,min_c 11.974 -7.153
max_c,min_c 1.0 0.0


In [9]:
ind = tf.where(real_ang[:,0]>20)[:,0]
data = tf.gather(data,ind)
detectors_rub = tf.gather(detectors_rub,ind)
dt_bunlde_mask  = tf.gather(dt_bunlde_mask,ind)
data.shape,ind.shape,detectors_rub.shape,dt_bunlde_mask.shape

(TensorShape([150604, 6, 6, 4]),
 TensorShape([150604]),
 TensorShape([150604, 6, 6, 3]),
 TensorShape([150604, 6, 6, 2]))

In [10]:
n=int(0.9*len(data))
train = data[:n]
test =data[n:]
detectors_rub = detectors_rub[:n]
dt_bunlde_mask =dt_bunlde_mask[:n]
print(train.shape,detectors_rub.shape,dt_bunlde_mask.shape)

(135543, 6, 6, 4) (135543, 6, 6, 3) (135543, 6, 6, 2)


In [11]:
def renorming(data,norm_params):
    val=np.array(list(norm_params.values()))
    max_c=val[:,0]
    min_c=val[:,1]
    data = data*(max_c-min_c) + min_c
#     if (i==0 and log):
#         data = data[:,:,:,i].assign(tf.math.exp(data[:,:,:,i])-1)
#     #         elif (i==1 or i==2):
#     #             data[:,:,:,i]=data[:,:,:,i]#*1e6
    return data

In [12]:
def norming_params(param):
    t = tf.reduce_max(param)
    b = tf.reduce_min(param)
    param = (param-b)/(t-b)
    return param,t,b
def renorm_params(param,p_t,p_b):
    return param*(p_t-p_b)+p_b

In [14]:
boundaries = [300]
values = [ 0.001,  0.001/5]
learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)
imp.reload(reco)

<module 'reconstruction' from '/home/rfit/Telescope_Array/Parameters_all/src/reconstruction.py'>

In [15]:
generator=tf.keras.models.load_model('../Models/Conditional+Phys_loss/save_model/generator/ep'+str(120))
discriminator=tf.keras.models.load_model('../Models/Conditional+Phys_loss/save_model/discriminator/ep'+str(120))

In [17]:
generator.summary()

Model: "Generator_model_"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               25600     
_________________________________________________________________
batch_normalization_24 (Batc (None, 512)               2048      
_________________________________________________________________
tf.nn.gelu_24 (TFOpLambda)   (None, 512)               0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 4, 4, 32)          0         
_________________________________________________________________
C_0 (Conv2D)                 (None, 4, 4, 32)          4096      
_________________________________________________________________
batch_normalization_25 (Batc (None, 4, 4, 32)     

In [16]:
def reco_loss(data,detectors_rub,dt_bunlde_mask,learning_rate_fn,norm_params):
    data_ = renorming(data,norm_params)
    chi_list, params_list = reco.optimization_2(data_,iterats=500,num=None,
                                          detectors_rub=detectors_rub,
                                          add_mask = None,
                                          use_L = False,
                                          use_core =False,
#                                           S800_rub=S800,
                                          optim_name="SGD",l_r =learning_rate_fn,
#                                           use_L3=False,
                                            find_core=True, flat_chanal  = False,
                                         )
    return chi_list[:,0]

In [18]:
noise = tf.random.normal((2000,50))
fake_data = generator(noise)
fake_data.shape

TensorShape([2000, 6, 6, 4])

In [36]:
imp.reload(reco)

<module 'reconstruction' from '/home/rfit/Telescope_Array/Parameters_all/src/reconstruction.py'>

In [37]:
data_ = renorming(fake_data,norm_params)
reco_flat = reco.optimization_flat(data_,iterats=500,num=None,
                                      detectors_rub=detectors_rub[:2000],
                                      add_mask = dt_bunlde_mask[:2000],
                                      use_L = False,
                                      use_core =False,
#                                           S800_rub=S800,
                                      optim_name="SGD",l_r =learning_rate_fn,
#                                           use_L3=False,
                                        find_core=True,
                                     )
reco_flat.shape

TensorShape([2000, 6, 6, 1])

In [39]:
err = tf.math.abs(data_[:,:,:,1:2] - reco_flat)
err_m = tf.where(fake_data[:,:,:,3:4]>0.5,err,0)
print(err_m[...,-1])

tf.Tensor(
[[[0.         0.         0.         1.7541013  0.         0.        ]
  [0.         0.         1.1096351  0.46492743 2.3702297  0.        ]
  [0.         0.         0.6894782  0.42121682 0.07137965 0.        ]
  [0.         0.         0.30692458 0.11491656 0.         0.        ]
  [0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.        ]]

 [[0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.        ]
  [0.         0.                nan        nan        nan 0.        ]
  [0.                nan        nan        nan        nan 0.        ]
  [0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.        ]]

 [[0.         0.         0.         0.         0.         0.        ]
  [0.         0.         1.0071539  0.59619325 2.5784693  0.        ]
  [0.

In [40]:
data_[1,:,:,1]*

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[-0.06047058, -0.1671648 ,  0.17111492, -0.17602444, -0.01586914,
         0.16425514],
       [-0.1880598 ,  0.13150692,  0.13606358, -0.18657684, -0.01683712,
        -0.0160265 ],
       [-0.13200855,  0.37557888, -1.7834816 ,  1.2913952 ,  3.1961813 ,
        -0.02818871],
       [ 0.15339565, -4.1131263 , -2.429635  ,  0.20509434,  2.9775133 ,
         0.03274441],
       [-0.35533333, -0.15544224, -0.31807518, -0.07409096,  0.04534721,
         0.1338377 ],
       [-0.10626221,  0.09365177,  0.08297253,  0.07596207, -0.03923035,
        -0.14170265]], dtype=float32)>